<a href="https://colab.research.google.com/github/CallMeL/TSP-Dynamic/blob/master/TSP_Dynamic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## lib

In [ ]:
import numpy as np
import math
import time

## tool Function & Class

In [ ]:
import pandas
import numpy as np
import math
import matplotlib.pyplot as plt 

In [ ]:
class Node:
	"""
	类名：Node
	函数功能：	从外界读取城市数据并处理
		输入	无
		输出	1 Position：各个城市的位置矩阵
			2 CityNum：城市数量
			3 Dist：城市间距离矩阵
	其他说明：无
	"""
	def __init__(self,CityNum):
		"""
		函数名：GetData()
		函数功能：	从外界读取城市数据并处理
			输入	无
			输出	1 Position：各个城市的位置矩阵
				2 CityNum：城市数量
				3 Dist：城市间距离矩阵
		其他说明：无
		"""
		self.visited=[False]*CityNum    #记录城市是否走过
		self.start=0                    #起点城市
		self.end=0                      #目标城市
		self.current=0                  #当前所处城市
		self.num=0                      #走过的城市数量
		self.pathsum=0                  #走过的总路程
		self.lb=0                       #当前结点的下界
		self.listc=[]                   #记录依次走过的城市

In [ ]:
def GetData(datapath):
	"""
	函数名：GetData()
	函数功能：	从外界读取城市数据并处理
		输入	无
		输出	1 Position：各个城市的位置矩阵
			2 CityNum：城市数量
			3 Dist：城市间距离矩阵
	其他说明：无
	"""
	dataframe = pandas.read_csv(datapath,sep=" ",header=None)
	Cities = dataframe.iloc[:,1:3]
	Position= np.array(Cities)				#从城市A到B的距离矩阵
	CityNum=Position.shape[0]				#CityNum:代表城市数量
	Dist = np.zeros((CityNum,CityNum))		#Dist(i,j)：城市i与城市j间的距离

	#计算距离矩阵
	for i in range(CityNum):
		for j in range(CityNum):
			if i==j:
				Dist[i,j] = math.inf
			else:
				Dist[i,j] = math.sqrt(np.sum((Position[i,:]-Position[j,:])**2))
	return Position,CityNum,Dist

In [ ]:
def ResultShow(Min_Path,BestPath,CityNum,string):
	"""
	函数名：GetData()
	函数功能：	从外界读取城市数据并处理
		输入	无
		输出	1 Position：各个城市的位置矩阵
			2 CityNum：城市数量
			3 Dist：城市间距离矩阵
	其他说明：无
	"""
	print("基于"+string+"求得的旅行商最短路径为：")
	for m in range(CityNum):
		print(str(BestPath[m])+"—>",end="")
	print(BestPath[CityNum])
	print("总路径长为："+str(Min_Path))
	print()

def draw(BestPath,Position,title):
	"""
	函数名：draw(BestPath,Position,title)
	函数功能：	通过最优路径将旅行商依次经过的城市在图表上绘制出来
		输入	1 	BestPath：最优路径
			2	Position：各个城市的位置矩阵
			3	title:图表的标题
		输出	无
	其他说明：无
	"""
	plt.title(title) 
	plt.plot(Position[:,0],Position[:,1],'bo')
	for i,city in enumerate(Position): 
		plt.text(city[0], city[1], str(i)) 
	plt.plot(Position[BestPath, 0], Position[BestPath, 1], color='red') 
	plt.show()

## Algorithm Functions

In [ ]:
def DP_recursion(city_start,citylists):
	"""
		函数名：DP_recursion(city_start,citylists)
		函数功能：  基于递归调用的动态规划算法核心
			输入	1: city_start:旅行商起始城市
				2: citylists:旅行商已经遍历过的城市，其中第i位为1代表城市i已经遍历过
	           		 第i位为0则代表城市i没有遍历(位数从0开始，第0位即最低位代表城市0)
			输出	1: dp_dist[city_start][citylists]:从城市city_start出发
	           		 遍历citylists内包含的各个城市的所花费的最小距离
		其他说明：无
	"""
	#判断是否已经求出从城市city出发遍历citylists内的城市的最短距离
	if IsSolvedMinDist(city_start,citylists):
		return dp_dist[city_start][citylists]
	
	#判断如果只遍历一个出发点城市，是则返回起始城市到开始城市的距离
	if IsOnlyExistCityN(city_init,citylists):
		return Dist[city_init][city_start]
	
	#求解T(vi,V)=min{Dij+T(vj,V-{Vj})},Vj属于V,公式
	dist_sum=math.inf
	for city in range(CityNum):
		if IsVisited(city,citylists):
			dist_temp=DP_recursion(city,Delete(city,citylists))+Dist[city_start][city]
			if dist_temp<dist_sum:
				dist_sum=dist_temp
				dp_path[city_start][citylists]=city
	dp_dist[city_start][citylists]=dist_sum
	
	return dp_dist[city_start][citylists]



In [ ]:
def DPMethond(CityNum,Dist,city_start):
	"""
		函数名：DPMethond(CityNum,Dist,city_start)
		函数功能： 动态规划算法的程序入口
			输入	1 CityNum：城市数量
				2 Dist：城市间距离矩阵
				3 city_start:旅行商起始城市
			输出	1 Min_Path：最优路径长
				2 BestPath：最优路径
		其他说明：无
	"""
	citylists = 2**(CityNum)-1          #初始已遍历城市列表为全部遍历
	
	Min_Path=DP_recursion(city_start,citylists)     #调用动态规划算法
	
	#此块代码块是根据dp_path矩阵找到最优路径
	BestPath=[]
	for i in range(CityNum):
		BestPath.append(city_start)
		city_start=int(dp_path[city_start][citylists])
		citylists=citylists&(~(1<<city_start))
	BestPath.append(city_init)
	return Min_Path,BestPath



def IsSolvedMinDist(city_start,citylists):
	"""
		函数名：IsSolvedMinDist(city_start,citylists)
		函数功能：判断是否已经求出了从城市city_start出发遍历citylists内的城市的最短距离
			输入	1: city_start:旅行商起始城市
				2: citylists:旅行商已经遍历过的城市，其中第i位为1代表城市i已经遍历过
	            第i位为0则代表城市i没有遍历(位数从0开始，第0位即最低位代表城市0)
			输出	1: 是——返回True，否——返回False
		其他说明：无
	"""
	return True if dp_dist[city_start][citylists] != -1 else False


def IsOnlyExistCityN(cityn,citylists):
	"""
		函数名：IsOnlyExistCityN(cityn,citylists)
		函数功能：判断如果只遍历一个出发点城市，是则返回起始城市到出发点城市的距离
			输入	1: city_start:旅行商起始城市
				2: citylists:旅行商已经遍历过的城市，其中第i位为1代表城市i已经遍历过
	            第i位为0则代表城市i没有遍历(位数从0开始，第0位即最低位代表城市0)
			输出	1: 是——返回True，否——返回False
		其他说明：无
	"""
	return True if citylists==2**cityn else False


def IsVisited(city,citylists):
	"""
		函数名：IsVisited(city,citylists)
		函数功能：判断城市city是否在遍历过的城市列表citylists中
			输入	1: city_start:旅行商起始城市
				2: citylists:旅行商已经遍历过的城市，其中第i位为1代表城市i已经遍历过
	            第i位为0则代表城市i没有遍历(位数从0开始，第0位即最低位代表城市0)
			输出	1: 是——返回True，否——返回False
		其他说明：无
	"""
	return True if citylists&(1<<city) else False


def Delete(city,citylists):
	"""
		函数名：Delete(city,citylists)
		函数功能：从遍历过的城市列表citylists中删去城市city
			输入	1: city_start:旅行商起始城市
				2: citylists:旅行商已经遍历过的城市，其中第i位为1代表城市i已经遍历过
	            第i位为0则代表城市i没有遍历(位数从0开始，第0位即最低位代表城市0)
			输出	1: 删去城市city后的已遍历过城市列表citylists
		其他说明：无
	"""
	return citylists&(~(1<<city))


In [ ]:
!wget https://github.com/Greatpanc/-TSP-/tree/master/data/TSP25cities.tsp

--2022-10-22 02:39:13--  https://github.com/Greatpanc/-TSP-/tree/master/data/TSP25cities.tsp
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/Greatpanc/-TSP-/blob/master/data/TSP25cities.tsp [following]
--2022-10-22 02:39:13--  https://github.com/Greatpanc/-TSP-/blob/master/data/TSP25cities.tsp
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘TSP25cities.tsp.1’

TSP25cities.tsp.1       [ <=>                ] 146.85K  --.-KB/s    in 0.02s   

2022-10-22 02:39:14 (6.31 MB/s) - ‘TSP25cities.tsp.1’ saved [150373]



## main function

结果：
贪心法求得最短旅行商经过所有城市回到原城市的最短路径为：
0->4->6->7->1->3->2->5->8->9->0
总路径长为：10127.552143541276

程序的运行时间是：0.064915142


In [ ]:
Position,CityNum,Dist = GetData("/content/TSP5cities.tsp")

city_init=0                         #起始城市

#dp_path[city][citylists]代表citylists内与city相连的城市信息
dp_path = np.ones((CityNum,2**CityNum))

#dp_dist[city,citylist]代表从城市city出发经过citylists内走过的城市后返回到city_start的最短距离矩阵
dp_dist = np.ones((CityNum,2**CityNum))*-1

start = time.clock()				#程序计时开始
Min_Path,BestPath=DPMethond(CityNum,Dist,city_init)	#调用动态规划算法
end = time.clock()					#程序计时结束

print()
ResultShow(Min_Path,BestPath,CityNum,"动态规划法")
print("程序的运行时间是：%s"%(end-start))
draw(BestPath,Position,"Dynamic Programming Method")
"""
结果：
贪心法求得最短旅行商经过所有城市回到原城市的最短路径为：
0->4->6->7->1->3->2->5->8->9->0
总路径长为：10127.552143541276

程序的运行时间是：0.064915142
"""